# PREDICTING SEPSIS RISK DURING IN-PATIENT ADMISSIONS
*Client: Royal Perth Hospital*

*Team: Group 7*

# Readme
The following libraries need to be installed in order to run the source code.

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import os

# 1.Dataset Processing

Load the dataset. The following tables are loaded for this project:
- PATIENTS
- ADMISSIONS
- DIAGNOSES_ICD
- LABEVENTS
- MICROBIOLOGYEVENTS
- D_ICD_DIAGNOSES
- D_ITEMS

In [ ]:
# load transaction dataset
df_patient = pd.read_csv('data\PATIENTS.csv')
df_admission = pd.read_csv('data\ADMISSIONS.csv')
df_diagnosis = pd.read_csv('data\DIAGNOSES_ICD.csv')
df_labvent = pd.read_csv('data\LABEVENTS.csv')
df_microbiology = pd.read_csv('data\MICROBIOLOGYEVENTS.csv')

# load description tables
df_desc_icd = pd.read_csv('data\D_ICD_DIAGNOSES.csv')
df_desc_item = pd.read_csv('data\D_ITEMS.csv')

## 1.1 Data Cleaning

## 1.2 Data Exploration

## 1.3 Feature Selection

# 2.Sepsis Risk Predicting Model Implementation

## 2.1 Logistics Regression Model

## 2.2 Random Forest Model

## 2.3 Gradient Boosted Model

## 2.4 LSTM Model

## 2.5 LSTM + Attention Model

## 2.6 Clustering Model

# 3.Model Testing

# 4.Local and Global Feature Explanation